## Importing Modules

In [25]:
import pandas as pd
import numpy as np
import os

# !! Important !! Change The Country Name Accordingly

In [26]:
country = "USA"

In [27]:
csv_files = [f for f in os.listdir() if f.endswith('.csv')]
csv_files

['Vistra Energy Stock Price History.csv',
 'Targa Resources Stock Price History.csv',
 'Diamondback Stock Price History.csv',
 'Baker Hughes Stock Price History.csv',
 'American Electric Power Stock Price History.csv',
 'Marathon Petroleum Stock Price History.csv',
 'Sempra Energy Stock Price History.csv',
 'Hess Stock Price History.csv',
 'MPLX LP Stock Price History.csv',
 'Valero Energy Stock Price History.csv',
 'Phillips 66 Stock Price History.csv',
 'Schlumberger Stock Price History.csv',
 'Dominion Energy Stock Price History.csv',
 'Occidental Stock Price History.csv',
 'Cheniere Energy Stock Price History.csv']

In [28]:
for csv in csv_files:
    df = pd.read_csv(csv)

    # Ensure the 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year


    # Filter for years 2014-2023
    df = df[(df['Year'] >= 2014) & (df['Year'] <= 2023)]
    df = df[(df['Year'] >= 2014) & (df['Year'] <= 2023)]
    df.to_csv(csv, index=False)

# Percent Change

In [24]:
def percentage_change(df, index=False):
    df[f'% Change in {'Stock' if index == False else 'Index'} {'Prices' if index == False else 'Level'}'] = df['Price'].pct_change() * 100
    return df

# For Standard Deviation Data

In [16]:
def calculate_yearly_std(df, index=False):
    """
    Calculates the yearly standard deviation of stock prices and adds a new column.
    :param df: DataFrame containing 'Date' and 'Close' price columns.
    :return: DataFrame with an additional 'Std Dev of Stock Prices' column.
    """


    # Calculate yearly standard deviation
    std_dev = df.groupby('Year')['Price'].std()
        
    # Merge with original DataFrame
    df = df.merge(std_dev.rename(f'Std Dev of {'Stock' if index == False else 'Index'} {'Prices' if index == False else 'Level'}'), on='Year', how='left')

    if index == False:
        df = df.drop(['Date', 'Price', 'Vol.', 'Change %', 'Open', 'High', 'Low'], axis='columns').drop_duplicates(f'Std Dev of {'Stock' if index == False else 'Index'} {'Prices' if index == False else 'Level'}')
    else:
        df = df.drop(['Date', 'Price', '% Change in Index Level'], axis='columns').drop_duplicates(f'Std Dev of {'Stock' if index == False else 'Index'} {'Prices' if index == False else 'Level'}')
    
    return df.sort_index(ascending=False)

# For Yearly Covariance

In [17]:
def calculate_yearly_covariance(stock_df, index_df):
    """
    Calculates the yearly covariance between stock prices and index level.
    
    :param stock_df: DataFrame containing 'Date' and 'Close' price columns for the stock.
    :param index_df: DataFrame containing 'Date' and 'Close' price columns for the index.
    :return: DataFrame with yearly covariance values.
    """
    # Ensure 'Date' column is in datetime format
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    index_df['Date'] = pd.to_datetime(index_df['Date'])
    
    # Extract year
    stock_df['Year'] = stock_df['Date'].dt.year
    index_df['Year'] = index_df['Date'].dt.year

    # Filter for years 2014-2023
    stock_df = stock_df[(stock_df['Year'] >= 2014) & (stock_df['Year'] <= 2023)]
    index_df = index_df[(index_df['Year'] >= 2014) & (index_df['Year'] <= 2023)]

    # Merge stock and index data on 'Date'
    merged_df = stock_df[['Date', 'Year', 'Price']].merge(index_df[['Date', 'Year', 'Price']], on=['Date', 'Year'], suffixes=('_Stock', '_Index'))

    # Calculate yearly covariance
    yearly_cov =  merged_df.groupby('Year')[['Price_Stock', 'Price_Index']].apply(lambda x: x.cov().iloc[0, 1])

    yearly_cov = pd.DataFrame(yearly_cov)

    yearly_cov.rename(columns={yearly_cov.columns[0]: 'Covariance'}, inplace=True)

    # Convert to DataFrame
    cov_df = yearly_cov.reset_index()

    return cov_df

# Finding Beta & Risks

In [18]:
def calculate_beta_risks(df, index, merged_df):
    """
    Calculates the stock market beta from a DataFrame with two columns: 
    'Stock' and 'Index'. Returns a new DataFrame with an additional column 'Beta'.
    
    Parameters:
        df (pd.DataFrame): A DataFrame with 'Stock' and 'Index' price columns.
    
    Returns:
        pd.DataFrame: Original DataFrame with an added 'Beta' column.
    """

    yearly_cov = calculate_yearly_covariance(df, index)
    yearly_cov = yearly_cov['Covariance']

    # Calculating Beta and adding it as a new column (constant for all rows)
    merged_df['Beta'] = yearly_cov.div(merged_df['Std Dev of Index Level'] ** 2)

    # Calculate total risk (variance of stock returns)
    merged_df['Total Risk'] = merged_df['Std Dev of Stock Prices'] ** 2

    # Calculate systematic risk (Beta squared * Index Variance)
    merged_df['Systematic Risk'] = merged_df['Total Risk'] * (merged_df['Beta'] ** 2)

    # Calculate idiosyncratic risk (Total Risk - Systematic Risk)
    merged_df['Idiosyncratic Risk'] = merged_df['Total Risk'] - merged_df['Systematic Risk']

    return merged_df

# Existing Rows

In [19]:
def ensure_minimum_rows(df):
    """
    Ensures that a DataFrame has at least 10 rows. If it has fewer, it adds NaN rows 
    except for the 'Year' column, which is filled with missing years between 2014 and 2023.
    
    Parameters:
        df (pd.DataFrame): Must contain a 'Year' column.
    
    Returns:
        pd.DataFrame: DataFrame with at least 10 rows.
    """
    
    if 'Year' not in df.columns:
        raise ValueError("The DataFrame must contain a 'Year' column.")
    
    # Get existing years and count current rows
    existing_years = set(df['Year'])
    num_rows = len(df)
    
    if num_rows >= 10:
        return df  # No need to add rows
    
    # Find missing years within the range 2014-2023
    possible_years = set(range(2014, 2024))  # 2024 is exclusive
    missing_years = sorted(possible_years - existing_years)
    
    # Determine how many extra rows are needed
    rows_needed = 10 - num_rows
    extra_years = missing_years[:rows_needed]  # Take only required years
    
    # Create a DataFrame with NaN values in all other columns
    nan_rows = pd.DataFrame({col: np.nan for col in df.columns}, index=range(rows_needed))
    nan_rows['Year'] = extra_years  # Fill 'Year' column with missing years
    
    # Concatenate original and new rows
    df = pd.concat([df, nan_rows], ignore_index=True)
    df = df.sort_values(by='Year', ascending=True)
    return df


# Index Calculations

In [20]:
index = pd.read_csv('Index/Index.csv')
index = index.iloc[:, :2]
index['Price'] = index['Price'].str.replace(',', '', regex=True).astype(float)
index = percentage_change(index, index=True)
index_std = calculate_yearly_std(index, index=True)

KeyError: 'Year'

In [21]:
# Create a new Excel writer object
with pd.ExcelWriter(f"{country}_stddev.xlsx", engine="xlsxwriter") as writer:

    for csv_file in csv_files:

        # Reading in from CSV File
        temp = pd.read_csv(csv_file)

        # Convert 'Date' column to datetime
        temp['Date'] = pd.to_datetime(temp['Date'])

        # Format 'Date' column as Month Month Day Day Year Year (e.g., 'Feb 26 2025')
        temp['Date'] = temp['Date'].dt.strftime('%Y-%m-%d')

        # Standard Deviation of Stock
        df = calculate_yearly_std(temp)

        # Merging with Index
        df = pd.merge(df, index_std, on='Year', how='inner')

        # Calulating Beta & 3 Risks
        df = calculate_beta_risks(temp, index, df)

        # Ensuring 10 rows are there (2014 to 2023)
        df = ensure_minimum_rows(df)
        
        sheet_name = csv_file[:9] # Remove .csv to use as sheet name
        df.to_excel(writer, sheet_name=sheet_name, index=False)  # Write to a separate sheet

print(f"Excel file '{country}_stddev.xlsx' created successfully!")

ModuleNotFoundError: No module named 'xlsxwriter'

In [22]:
with pd.ExcelWriter(f"{country}_company.xlsx", engine="xlsxwriter") as writer:
    for csv_file in csv_files:
        # Read CSV and process stock data
        df = pd.read_csv(csv_file)
        df = percentage_change(df, index=False)
        
        # Convert Date columns to datetime (without converting to strings)
        df['Date'] = pd.to_datetime(df['Date'])
        index_copy = index.copy()
        index_copy['Date'] = pd.to_datetime(index_copy['Date'])

        # Select relevant columns
        df = df[['Date', 'Price', '% Change in Stock Prices']]
        index_copy = index_copy[['Date', 'Price', '% Change in Index Level']]
        
        # Merge using datetime columns
        df = pd.merge(df, index_copy, on='Date')
        
        # Rename columns
        df = df.rename(columns={'Price_x': 'Stock Price', 'Price_y': 'Index Level'})

        # Write to Excel
        sheet_name = csv_file.replace(".csv", "")[:9]  # Cleaner sheet name
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Excel file '{country}_company.xlsx' created successfully!")

ModuleNotFoundError: No module named 'xlsxwriter'